<a href="https://colab.research.google.com/github/EduBernardo/mba-projects/blob/master/Eduardo_Luiz_Bernardo_da_Silva_ES_23_Avalia%C3%A7%C3%A3o_Final_Analisando_dados_de_acidentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

## Avaliação final

## 1. Usando o Spark SQL, carregue os arquivos a seguir:

+ si-bol-2019.csv - Boletins de ocorrência de acidentes automobilisticos  - https://ckan.pbh.gov.br/dataset/6511cb66-3635-4560-95cc-d0c39aafb547/resource/ab6db535-b706-4e7b-9fdc-3bc1e823401e/download/si-bol-2019.csv

+ si_env-2019.csv - Dados sobre os envolvidos nos acidentes - https://ckan.pbh.gov.br/dataset/b127c1d8-9e1b-4820-884a-8bd8129ba5e3/resource/903286ca-b77f-44ad-aa6a-958aa018c33f/download/si_env-2019.csv
+ si_log-2019.csv - Dados sobre a localização dos acidentes - https://ckan.pbh.gov.br/dataset/a07412b3-8371-402a-8ca6-68a518bf2403/resource/ad490dd1-7af5-4868-b35f-9471f15be744/download/si-log-2019.csv

## 2. Crie uma tabela contendo os campos:

+ data_hora 
+ tipo_acidente 
+ condicao_tempo 
+ pavimento
+ severidade 
+ cinto_seguranca 
+ embreagues 
+ idade 
+ sexo
+ especie_veiculo  
+ nome_municipio
+ nome_bairro
+ tipo_logradouro 
+ nome_logradouro
+ total_envolvidos

## 3. Realize a limpeza dos dados, removendo: 
+ valores nulos
+ dados inválidos

## 4. Crie as seguintes consultas: 

1. Número de acidentes por mês/ano
2. Total de acidentes com vítima por bairro;
3. Total de acidentes por tipo de pavimento e condição do tempo;
4. Total de pessoas acidentadas por tipo de veiculo e pavimento;

## 5. Utilizando as biblioteca pandas e matplotlib, exiba o resultado dos itens 1 e 3 em forma de gráficos:



In [40]:
# carregando arquivos da tabela de Boletins

df_bol = spark.read.csv("si-bol-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela boletins contém os seguintes campos:
NUMERO_BOLETIM;
DATA_HORA_BOLETIM;
DATA_INCLUSAO;
TIPO_ACIDENTE;
DESC_TIPO_ACIDENTE;
COD_TEMPO;
DESC_TEMPO;
COD_PAVIMENTO;
PAVIMENTO;
COD_REGIONAL;
DESC_REGIONAL;
ORIGEM_BOLETIM;
LOCAL_SINALIZADO;
VELOCIDADE_PERMITIDA;
COORDENADA_X;
COORDENADA_Y;
HORA_INFORMADA;
INDICADOR_FATALIDADE;
VALOR_UPS;
DESCRIÇÃO_UPS;
DATA_ALTERACAO_SMSA;
VALOR_UPS_ANTIGA;
DESCRIÇÃO_UPS_ANTIGA;

In [41]:
# carregando arquivos da tabela de Localização

df_log = spark.read.csv("si-log-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela localização contém os seguintes campos:
Nº_boletim;
data_boletim;
 Nº_municipio;
nome_municipio;
seq_logradouros;
Nº_logradouro;
tipo_logradouro;
nome_logradouro;
tipo_logradouro_anterior;
nome_logradouro_anterior;
Nº_bairro;
nome_bairro;
tipo_bairro;
descricao_tipo_bairro;
Nº_imovel;
Nº_imovel_proximo;


In [42]:
# carregando arquivos da tabela de Envolvidos

df_env = spark.read.csv("si_env-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela envolvidos contém os seguintes campos:
num_boletim;
data_hora_boletim;
Nº_envolvido;
condutor;
cod_severidade;
desc_severidade;
sexo;
cinto_seguranca;
Embreagues;
Idade;
nascimento;
categoria_habilitacao;
descricao_habilitacao;
declaracao_obito;
cod_severidade_antiga;
especie_veiculo;
pedestre;
passageiro;

In [ ]:
# apresentando as tabelas
df_bol.show(100)
df_log.show(100)
df_env.show(100)

In [ ]:
df_bol.withColumnRenamed(" NUMERO_BOLETIM","NUMERO_BOLETIM").show()

In [ ]:
df_log.withColumnRenamed("Nº_boletim","NUMERO_BOLETIM").show()

In [ ]:
#Join entre tabela boletins e localização
df_join_bol_log = df_bol.join(df_log, df_bol[' NUMERO_BOLETIM'] == df_log['Nº_boletim'], 'left')
df_join_bol_log.show(10)

In [ ]:
#Join entre tabela resultante do join anterior + tabela envolvidos
df_join_total = df_join_bol_log.join(df_env, df_bol[' NUMERO_BOLETIM'] == df_env['num_boletim'], 'left')
df_join_total.show(10)

In [70]:
#Criando visão específica com os campos solicitados
import pyspark.sql.functions as F

df_join_total.groupby(' DATA HORA_BOLETIM',' TIPO_ACIDENTE', ' DESC_TIPO_ACIDENTE',' DESC_TEMPO',' PAVIMENTO',' desc_severidade',' cinto_seguranca',
                      ' Embreagues',' Idade',' sexo','especie_veiculo','nome_municipio','nome_bairro','tipo_logradouro','nome_logradouro').agg(F.sum(' Nº_envolvido').alias('total_envolvidos')).show(100,truncate=False)

+------------------+--------------+--------------------------------------------------+---------------+---------------+----------------+----------------+-------------+------+-----+------------------------------+------------------------------+--------------------------------------------------+---------------+--------------------------------------------------+----------------+
| DATA HORA_BOLETIM| TIPO_ACIDENTE| DESC_TIPO_ACIDENTE                               | DESC_TEMPO    | PAVIMENTO     | desc_severidade| cinto_seguranca| Embreagues  | Idade| sexo|especie_veiculo               |nome_municipio                |nome_bairro                                       |tipo_logradouro|nome_logradouro                                   |total_envolvidos|
+------------------+--------------+--------------------------------------------------+---------------+---------------+----------------+----------------+-------------+------+-----+------------------------------+------------------------------+-----